In [0]:
SET school.dataset_path=dbfs:/mnt/DE-Associate-Book/datasets/school;

create or refresh streaming table course_bronze
comment 'bronze raw courses data, ingested from CDC feed'
as
select * from cloud_files("dbfs:/mnt/DE-Associate-Book/datasets/school/courses-cdc");

create or refresh streaming table courses_silver
comment 'silver cleansed courses data';

apply changes into Live.courses_silver
from stream(Live.course_bronze)
keys(course_id)
apply as delete when row_status = 'DELETE'
sequence by row_time
columns * except(row_status,row_time);

create or replace materialized view instructor_counts_stats
comment 'Number of courses per instructor'
as
select instructor, count(*) courses_count,
      current_timestamp() update_time
 from Live.courses_silver 
group by instructor;

create temporary live view courses_sales
as select c.title,o.quantity
  from(select *,explode(courses) as course
  from Live.enrollments_cleaned) o
inner join Live.courses_silver c
  on o.course.course_id = c.course_id;


